In [112]:
#Rod Ta
#9/25/17

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.colors import ListedColormap

In [113]:
# read Iris data set

#df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
df = pd.read_csv('iris.csv', header = None)
df.tail()

IA = df.iloc[:,:].values
#for i_ in range(0,len(df),1):
#    print(i_, IA[i_,:])

Cl = IA[:,4]

In [114]:
# get Iris classifications

IC = np.unique(IA[:,4])
print (IC)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [115]:
pltC = ('red', 'blue', 'green')
pltM = ('o', 'x', '*')
for i in range(0,3):
    print(IC[i], pltC[i], pltM[i])

Iris-setosa red o
Iris-versicolor blue x
Iris-virginica green *


In [116]:
# setup Iris feature lablels

IrisF=("sepal length in cm", "sepal width in cm", "petal length in cm", "petal width in cm")
for i in range(0,4):
    print(IrisF[i])

sepal length in cm
sepal width in cm
petal length in cm
petal width in cm


In [117]:
class AdalineGD(object):
    """ADAptive LInear NEuron classifier.

    Code from "Python Machine Learning," Sebastian Raschka, Packt Publishing, 2015,
    with modifications.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.

    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    errors_ : list
        Number of misclassifications in every epoch.

    """
    def __init__(self, eta=0.01, n_iter=50):
        self.eta = eta
        self.n_iter = n_iter

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_input(self, X):
        """ Calculate net input """
        return (np.dot(X, self.w_[1:]) + self.w_[0])

    def activation(self, X):
        """ Compute linear activation """
        return self.net_input(X)

    def predict(self, X):
        """ Return class label after unit step """
        return np.where(self.activation(X) >= 0.0, 1, -1) 
    
    def accuracy(self, X, y):
        """ Return accuracy of the model """
        return (1.0 - abs(y - self.predict(X)).sum() / (2.0 * y.size))
        

In [118]:
TwoFeatureMean = []

In [119]:
def TwoFeature(iris1,iris2,feature1,feature2):
    # focus on classification of "virginica" and "versicolor", class indexes 0 and 1; 
    # set to 1 and -1 in class array
    cdx = np.array([iris1,iris2])   
    # focus on using only 2 features
    fdx = np.array([feature1,feature2])

    i = 0
    numCl = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:
            numCl += 1
        if Cl[i] == IC[cdx[1]]:
            numCl += 1    
        i += 1  

    NC = np.zeros((numCl))
    NF = np.zeros((numCl, 2))

    i = 0
    j = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:        
            NC[j] = 1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            j += 1
        if Cl[i] == IC[cdx[1]]:      
            NC[j] = -1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            j += 1
        i += 1
        
    NF_Std = np.copy(NF)
    NF_Std[:,0] = (NF[:,0] - NF[:,0].mean()) / NF[:,0].std()
    NF_Std[:,1] = (NF[:,1] - NF[:,1].mean()) / NF[:,1].std()
    
    ada = AdalineGD(n_iter=50, eta=0.003)
    ada.fit(NF_Std, NC)
    
    print (IC[iris1] + "," +IC[iris2] + " " + IrisF[feature1] + ", "+ IrisF[feature2] + ":") 
    print (ada.accuracy(NF_Std, NC))
    TwoFeatureMean.append(ada.accuracy(NF_Std, NC))

In [120]:
# two features
print ("Two Features: \n")

TwoFeature(1,2,0,1)
TwoFeature(1,2,0,2)
TwoFeature(1,2,0,3)
TwoFeature(1,2,1,2)
TwoFeature(1,2,1,3)
TwoFeature(1,2,2,3)
    
print ("\naverage accuracy: ", sum(TwoFeatureMean)/len(TwoFeatureMean))

Two Features: 

Iris-versicolor,Iris-virginica sepal length in cm, sepal width in cm:
0.75
Iris-versicolor,Iris-virginica sepal length in cm, petal length in cm:
0.94
Iris-versicolor,Iris-virginica sepal length in cm, petal width in cm:
0.94
Iris-versicolor,Iris-virginica sepal width in cm, petal length in cm:
0.93
Iris-versicolor,Iris-virginica sepal width in cm, petal width in cm:
0.95
Iris-versicolor,Iris-virginica petal length in cm, petal width in cm:
0.94

average accuracy:  0.908333333333


In [121]:
ThreeFeatureMean = []

In [122]:
def ThreeFeature(iris1,iris2,feature1,feature2,feature3):
    # focus on classification of "virginica" and "versicolor", class indexes 0 and 1; 
    # set to 1 and -1 in class array
    cdx = np.array([iris1,iris2])   
    # focus on using three features
    fdx = np.array([feature1,feature2,feature3])

    i = 0
    numCl = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:
            numCl += 1
        if Cl[i] == IC[cdx[1]]:
            numCl += 1    
        i += 1  

    NC = np.zeros((numCl))
    NF = np.zeros((numCl, 3))

    i = 0
    j = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:        
            NC[j] = 1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            NF[j,2] = IA[i,fdx[2]]
            j += 1
        if Cl[i] == IC[cdx[1]]:      
            NC[j] = -1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            NF[j,2] = IA[i,fdx[2]]
            j += 1
        i += 1
        
    NF_Std = np.copy(NF)
    NF_Std[:,0] = (NF[:,0] - NF[:,0].mean()) / NF[:,0].std()
    NF_Std[:,1] = (NF[:,1] - NF[:,1].mean()) / NF[:,1].std()
    NF_Std[:,2] = (NF[:,2] - NF[:,2].mean()) / NF[:,2].std()
    
    ada = AdalineGD(n_iter=50, eta=0.003)
    ada.fit(NF_Std, NC)
    
    print (IC[iris1] + "," +IC[iris2] + " " + IrisF[feature1] + ", "+ IrisF[feature2] + ", " + IrisF[feature3] + ":") 
    print (ada.accuracy(NF_Std, NC))
    ThreeFeatureMean.append(ada.accuracy(NF_Std, NC))

In [123]:
# three features
print ("Three features: \n" )

ThreeFeature(1,2,0,1,2)
ThreeFeature(1,2,0,1,3)
ThreeFeature(1,2,0,2,3)
ThreeFeature(1,2,1,2,3)

print ("\naverage accuracy: ", sum(ThreeFeatureMean)/len(ThreeFeatureMean))

Three features: 

Iris-versicolor,Iris-virginica sepal length in cm, sepal width in cm, petal length in cm:
0.93
Iris-versicolor,Iris-virginica sepal length in cm, sepal width in cm, petal width in cm:
0.95
Iris-versicolor,Iris-virginica sepal length in cm, petal length in cm, petal width in cm:
0.96
Iris-versicolor,Iris-virginica sepal width in cm, petal length in cm, petal width in cm:
0.96

average accuracy:  0.95


In [124]:
FourFeatureMean = []

In [125]:
def FourFeature(iris1,iris2,feature1,feature2,feature3,feature4):
    # focus on classification of "virginica" and "versicolor", class indexes 0 and 1; 
    # set to 1 and -1 in class array
    cdx = np.array([iris1,iris2])   
    # focus on using all 4 features
    fdx = np.array([feature1,feature2,feature3,feature4])

    i = 0
    numCl = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:
            numCl += 1
        if Cl[i] == IC[cdx[1]]:
            numCl += 1    
        i += 1  

    NC = np.zeros((numCl))
    NF = np.zeros((numCl, 4))

    i = 0
    j = 0
    for i in range(0 , len(Cl)):
        if Cl[i] == IC[cdx[0]]:        
            NC[j] = 1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            NF[j,2] = IA[i,fdx[2]]
            NF[j,3] = IA[i,fdx[3]]
            j += 1
        if Cl[i] == IC[cdx[1]]:      
            NC[j] = -1
            NF[j,0] = IA[i,fdx[0]]
            NF[j,1] = IA[i,fdx[1]]
            NF[j,2] = IA[i,fdx[2]]
            NF[j,3] = IA[i,fdx[3]]
            j += 1
        i += 1
        
    NF_Std = np.copy(NF)
    NF_Std[:,0] = (NF[:,0] - NF[:,0].mean()) / NF[:,0].std()
    NF_Std[:,1] = (NF[:,1] - NF[:,1].mean()) / NF[:,1].std()
    NF_Std[:,2] = (NF[:,2] - NF[:,2].mean()) / NF[:,2].std()
    NF_Std[:,3] = (NF[:,3] - NF[:,3].mean()) / NF[:,3].std()
    
    ada = AdalineGD(n_iter=50, eta=0.003)
    ada.fit(NF_Std, NC)
    
    print (IC[iris1] + "," +IC[iris2] + " " + IrisF[feature1] + ", "+ IrisF[feature2] + ", " + IrisF[feature3] + ", " + IrisF[feature4] + ":") 
    print (ada.accuracy(NF_Std, NC))
    FourFeatureMean.append(ada.accuracy(NF_Std, NC))

In [126]:
#Four features
FourFeature(1,2,0,1,2,3)
print ("\naverage accuracy: ", sum(FourFeatureMean)/len(FourFeatureMean))

Iris-versicolor,Iris-virginica sepal length in cm, sepal width in cm, petal length in cm, petal width in cm:
0.97

average accuracy:  0.97


In [127]:
'''
The best accuracy I was able to achieve from the 11 classes is the class using all 4 features
at once

In the case of using two features at a time:
    -Highest accruacy obtained: 0.95
    -Average accuracy: 0.908333333333
    
In the case of using three features at a time:
    -Highest accruacy obtained: 0.96
    -Average accuracy: 0.95
    
In the case of using four features at a time:
    -Highest accuracy obtained: 0.97
    -Average accuracy: 0.97 

Through my findings mentioned above, using more dimensions definitely helps when tying to
classify the data in this dataset. There is a strong correlation with the increase in accuracy
and the use of more dimensions.
'''

'\nThe best accuracy I was able to achieve from the 11 classes is the class using all 4 features\nat once\n\nIn the case of using two features at a time:\n    -Highest accruacy obtained: 0.95\n    -Average accuracy: 0.908333333333\n    \nIn the case of using three features at a time:\n    -Highest accruacy obtained: 0.96\n    -Average accuracy: 0.95\n    \nIn the case of using four features at a time:\n    -Highest accuracy obtained: 0.97\n    -Average accuracy: 0.97 \n\nThrough my findings mentioned above, using more dimensions definitely helps when tying to\nclassify the data in this dataset. There is a strong correlation with the increase in accuracy\nand the use of more dimensions.\n'